In [ ]:
import math
import argparse
import os
import re
import pandas as pd
pd.options.mode.chained_assignment=None
from tqdm.auto import tqdm
tqdm.pandas(position=0,leave=True)
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import numpy as np
from sklearn import metrics
import utils

import joblib

import matplotlib.pyplot as plt
import matplotlib.ticker as mtick

import nltk
# from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk_data_dir=os.path.join("/opt/omniai/work/instance1/jupyter/", "transformers-models","nltk_data")
stopwords_file = open(nltk_data_dir + '/corpora/stopwords/english')
stopwords_list = stopwords_file.readlines()
STOPWORDS=[x.strip() for x in stopwords_list]
nltk.data.path.append(nltk_data_dir)

In [ ]:
data_name="inference_data_0923"
path_0="/opt/omniai/work/instance1/jupyter/v5_new_email/cb-cx-complaints-container/model_monitoring"
email_data=pd.read_pickle(os.path.join(path_0,data_name))
test_output_v0=utils.model_evaluate(email_data["target"].values.reshape(-1),email_data.prob_pred.squeeze(),email_data.best_threshold.unique()[0])
# test_output_v0

In [ ]:
data_name="inf_0923_feedback"
dedup_type="dedup_target"
path=f"/opt/omniai/work/instance1/jupyter/v5_new_email/TFIDF_revisit_v1/daily_inference/split_data/pred_output/{dedup_type}/"
email_data=pd.read_pickle(os.path.join(path,data_name))

data_name="inf_0823_feedback"
path=f"/opt/omniai/work/instance1/jupyter/v5_new_email/TFIDF_revisit_v1/daily_inference/reference_data/split_data/pred_output/{dedup_type}/"
best_threshold=pd.read_pickle(os.path.join(path,data_name)).best_threshold.unique()[0]
# model_dir=f"/opt/omniai/work/instance1/jupyter/v5_new_email/TFIDF_revisit_v1/features/{dedup_type}/tfidf_model/"
# with open(os.path.join(model_dir,"best_threshold_val.txt"),'r') as f:
#     best_threshold=float(f.readline())
test_output_v1=utils.model_evaluate(email_data["target"].values.reshape(-1),email_data.prob_pred.squeeze(),best_threshold)

In [ ]:
data_name="inf_0923"
model="longformer_base_4096"
path=f"/opt/omniai/work/instance1/jupyter/v5_new_email/Fine-Tuning_v1/daily_inference/split_data/pred_output/{model}/"
email_data=pd.read_pickle(os.path.join(path,data_name))

data_name="inf_0823"
path=f"/opt/omniai/work/instance1/jupyter/v5_new_email/Fine-Tuning_v1/daily_inference/reference_data/split_data/pred_output/{model}/"
best_threshold=pd.read_pickle(os.path.join(path,data_name)).best_threshold.unique()[0]

email_data.dropna(subset=["target"],axis=0,inplace=True)
test_output_v2=utils.model_evaluate(email_data["target"].values.reshape(-1),email_data.Predicted_prob.squeeze(),best_threshold)
# test_output_v2

In [ ]:
a, b=email_data["target"].value_counts().tolist()
precision_lexican=(b/(a+b))
    
print()
print("********** precision comparision ***********")
print()
print("{:<25}{:<20.4%}".format("Lexican Search",precision_lexican))
print("{:<25}{:<20.4%}".format("existing TFIDF Model",test_output_v0["precision"]))
print("{:<25}{:<20.4%}".format("updated TFIDF Model",test_output_v1["precision"]))
print("{:<25}{:<20.4%}".format("longformer-base ",test_output_v2["precision"]))

In [ ]:
print()
print("********** Recall comparision ***********")
print()
print("{:<25}{:<20.0%}".format("Lexican Search",1))
print("{:<25}{:<20.4%}".format("existing TFIDF Model",test_output_v0["recall"]))
print("{:<25}{:<20.4%}".format("updated TFIDF Model",test_output_v1["recall"]))
print("{:<25}{:<20.4%}".format("longformer-base ",test_output_v2["recall"]))

In [ ]:
a, b=email_data["target"].value_counts().tolist()

print()
print("********** False Positive ***********")
print()
print("{:<25}{:<20,}".format("Lexican Search",a))
print("{:<25}{:<20,}".format("existing TFIDF Model",test_output_v0["false positive"]))
print("{:<25}{:<20,}".format("updated TFIDF Model",test_output_v1["false positive"]))
print("{:<25}{:<20,}".format("longformer-base ",test_output_v2["false positive"]))

In [ ]:
print()
print("********** False Negative ***********")
print()
print("{:<20}{:<20,}".format("Lexican Search",0))
print("{:<25}{:<20,}".format("existing TFIDF Model",test_output_v0["false_negative"]))
print("{:<25}{:<20,}".format("updated TFIDF Model",test_output_v1["false_negative"]))
print("{:<25}{:<20,}".format("longformer-base ",test_output_v2["false_negative"]))

In [ ]:
print()
print("********** Reduce Lexican Hit Percentage ***********")
print()
lexican_hit=a+b
x0=test_output_v0['false positive']+int(test_output_v0['total positive']*test_output_v0['recall'])
x1=test_output_v1['false positive']+int(test_output_v1['total positive']*test_output_v1['recall'])
x2=test_output_v2['false positive']+int(test_output_v2['total positive']*test_output_v2['recall'])

print("{:<25}{:<20.2%}".format("existing TFIDF Model", 1-x0/lexican_hit))
print("{:<25}{:<20.2%}".format("updated TFIDF Model", 1-x1/lexican_hit))
print("{:<25}{:<20.2%}".format("longformer-base ",  1-x2/lexican_hit))

In [ ]:
data_name="inference_data_0923"
path_0="/opt/omniai/work/instance1/jupyter/v5_new_email/cb-cx-complaints-container/model_monitoring"
email_data_v0=pd.read_pickle(os.path.join(path_0,data_name))
keep_columns=['snapshot_id', 'gcid', 'time','target','prob_pred','best_threshold']
email_data_v0=email_data_v0.loc[:,keep_columns]
email_data_v0.rename(columns={'prob_pred':'prob_pred_v0','best_threshold':'best_threshold_v0'},inplace=True)
email_data_v0['time'] = pd.to_datetime(email_data_v0['time'])

In [ ]:
data_name="inf_0923_feedback"
dedup_type="dedup_target"
path=f"/opt/omniai/work/instance1/jupyter/v5_new_email/TFIDF_revisit_v1/daily_inference/split_data/pred_output/{dedup_type}/"
email_data_v1=pd.read_pickle(os.path.join(path,data_name))

data_name="inf_0823_feedback"
path=f"/opt/omniai/work/instance1/jupyter/v5_new_email/TFIDF_revisit_v1/daily_inference/reference_data/split_data/pred_output/{dedup_type}/"
best_threshold=pd.read_pickle(os.path.join(path,data_name)).best_threshold.unique()[0]

email_data_v1['best_threshold_v1']=best_threshold
keep_columns=['snapshot_id', 'gcid', 'time','target','prob_pred','best_threshold_v1']
email_data_v1=email_data_v1.loc[:,keep_columns]
email_data_v1.rename(columns={'prob_pred':'prob_pred_v1'},inplace=True)

In [ ]:
data_name="inf_0923"
model="longformer_base_4096"
path=f"/opt/omniai/work/instance1/jupyter/v5_new_email/Fine-Tuning_v1/daily_inference/split_data/pred_output/{model}/"
email_data_v2=pd.read_pickle(os.path.join(path,data_name))

data_name="inf_0823"
path=f"/opt/omniai/work/instance1/jupyter/v5_new_email/Fine-Tuning_v1/daily_inference/reference_data/split_data/pred_output/{model}/"
best_threshold=pd.read_pickle(os.path.join(path,data_name)).best_threshold.unique()[0]

email_data_v2['best_threshold_v2']=best_threshold
keep_columns=['snapshot_id', 'gcid', 'time','target','Predicted_prob','best_threshold_v2']
email_data_v2=email_data_v2.loc[:,keep_columns]
email_data_v2.rename(columns={'Predicted_prob':'prob_pred_v2'},inplace=True)

In [ ]:
email_data=email_data_v0.merge(email_data_v1, on=['snapshot_id', 'gcid','time','target'],how="inner")
email_data=email_data.merge(email_data_v2, on=['snapshot_id', 'gcid','time','target'],how="inner")
email_data['year'] = email_data.time.apply(lambda x: x.year)
email_data['month'] = email_data.time.apply(lambda x: x.month)
email_data['day'] = email_data.time.apply(lambda x: x.day)

In [ ]:
def recall_precision_chart(email_data, var="target", subtitle="test_set=0901~0930"):
    recall_v0=[]
    recall_v1=[]
    recall_v2=[]
    precision_v0=[]
    precision_v1=[]
    precision_v2=[]

    recall_lexican=[]
    precision_lexican=[]

    daily_vt=sorted(email_data.day.unique())

    for i,t in enumerate(daily_vt):
        tempt=email_data[email_data.day==t]    
        best_threshold_v0=tempt.best_threshold_v0.unique()[0]
        best_threshold_v1=tempt.best_threshold_v1.unique()[0]
        best_threshold_v2=tempt.best_threshold_v2.unique()[0]

        recall_lexican.append(1)

        if tempt[tempt[var]==1].shape[0]!=0:
            a, b=tempt[var].value_counts().tolist()
            precision_lexican.append(b/(a+b))

            test_output_v0=utils.model_evaluate(tempt[var].values.reshape(-1),tempt.prob_pred_v0.squeeze(),best_threshold_v0)
            recall_v0.append(test_output_v0["recall"])
            precision_v0.append(test_output_v0["precision"])

            test_output_v1=utils.model_evaluate(tempt[var].values.reshape(-1),tempt.prob_pred_v1.squeeze(),best_threshold_v1)
            recall_v1.append(test_output_v1["recall"])
            precision_v1.append(test_output_v1["precision"])

            test_output_v2=utils.model_evaluate(tempt[var].values.reshape(-1),tempt.prob_pred_v2.squeeze(),best_threshold_v2)
            recall_v2.append(test_output_v2["recall"])
            precision_v2.append(test_output_v2["precision"])

        else:
            recall_v0.append(1)
            recall_v1.append(1)
            recall_v2.append(1)
            precision_v0.append(0)
            precision_v1.append(0)
            precision_v2.append(0)
            precision_lexican.append(0)
            
    # models = ['Lexican_search','TFIDF_lightgbm_v0','TFIDF_lightgbm_v1','TFIDF_lightgbm_v2']
    # markers = ['o', 's','x','D']
    # colors = ['blue','red','green', 'purple']
    
    models = ['Lexican_search','Existing TFIDF Model','Updated TFIDF Model','Language Model']
    markers = ['o', 's','x','D']
    colors = ['blue','red','green', 'purple']
    
    # Plot precision and recall
    plt.figure(figsize=(10, 8))

    plt.plot(daily_vt, precision_lexican, marker=markers[0],  color=colors[0], label=models[0], linewidth=3, linestyle=":", markersize=8)
    plt.plot(daily_vt, precision_v0, marker=markers[1],  color=colors[1], label="Existing TFIDF Model", linewidth=3, linestyle=":", markersize=8)
    plt.plot(daily_vt, precision_v1, marker=markers[2],  color=colors[2], label="Updated TFIDF Model", linewidth=3, linestyle=":", markersize=8)
    plt.plot(daily_vt, precision_v2, marker=markers[3],  color=colors[3], label="Language Model", linewidth=3, linestyle=":", markersize=8)

    plt.xlabel('Day', fontsize=14)
    plt.ylabel('Precision', fontsize=14)
    plt.title(f'Daily Precision \n({subtitle})', fontsize=16)
    plt.grid(True)

    # Format axis values as percentages
    ax = plt.gca()
    ax.yaxis.set_major_locator(mtick.MultipleLocator(base=0.001))
    ax.yaxis.set_major_formatter(mtick.PercentFormatter(xmax=1, decimals=2))

    # plt.ylim(0.006,0.03)
    # plt.xlim(0.96,1.001)

    plt.xticks(daily_vt)
    plt.xticks(rotation=90)
    # Set the legend
    plt.legend(loc='upper right', fontsize=16)
    plt.show()
    
    models = ['Lexican_search','Existing TFIDF Model','Updated TFIDF Model','Language Model']
    markers = ['o', 's','x','D']
    colors = ['blue','red','green', 'purple']

    # Plot precision and recall
    plt.figure(figsize=(10, 8))

    plt.plot(daily_vt, recall_lexican, marker=markers[0],  color=colors[0], label=models[0], linewidth=3, linestyle=":", markersize=8)
    plt.plot(daily_vt, recall_v0, marker=markers[1],  color=colors[1], label="Existing TFIDF Model", linewidth=3, linestyle=":", markersize=8)
    plt.plot(daily_vt, recall_v1, marker=markers[2],  color=colors[2], label="Updated TFIDF Model", linewidth=3, linestyle=":", markersize=8)
    plt.plot(daily_vt, recall_v2, marker=markers[3],  color=colors[3], label="Language Model", linewidth=3, linestyle=":", markersize=8)

    plt.xlabel('Day', fontsize=14)
    plt.ylabel('Recall', fontsize=14)
    plt.title(f'Daily Recall \n({subtitle})', fontsize=16)
    plt.grid(True)

    # Format axis values as percentages
    ax = plt.gca()
    ax.yaxis.set_major_locator(mtick.MultipleLocator(base=0.05))
    ax.yaxis.set_major_formatter(mtick.PercentFormatter(xmax=1, decimals=2))

    plt.xticks(daily_vt)
    plt.xticks(rotation=90)

    # Set the legend
    plt.legend(loc='lower right', fontsize=16)
    plt.show()

In [ ]:
def false_psoitive_negative_chart(email_data, var="target", subtitle="test_set=0901~0930"):
    false_negative_v0=[]
    false_negative_v1=[]
    false_negative_v2=[]
    false_positive_v0=[]
    false_positive_v1=[]
    false_positive_v2=[]

    false_positive_lexican=[]
    daily_vt=sorted(email_data.day.unique())

    for i,t in enumerate(daily_vt):
        tempt=email_data[email_data.day==t]    
        best_threshold_v0=tempt.best_threshold_v0.unique()[0]
        best_threshold_v1=tempt.best_threshold_v1.unique()[0]
        best_threshold_v2=tempt.best_threshold_v2.unique()[0]    

        if tempt[tempt[var]==1].shape[0]!=0:
            a, b=tempt[var].value_counts().tolist()    
            false_positive_lexican.append(a)
            test_output_v0=utils.model_evaluate(tempt[var].values.reshape(-1),tempt.prob_pred_v0.squeeze(),best_threshold_v0)
            false_positive_v0.append(test_output_v0["false positive"])
            false_negative_v0.append(test_output_v0["false_negative"])

            test_output_v1=utils.model_evaluate(tempt[var].values.reshape(-1),tempt.prob_pred_v1.squeeze(),best_threshold_v1)
            false_positive_v1.append(test_output_v1["false positive"])
            false_negative_v1.append(test_output_v1["false_negative"])

            test_output_v2=utils.model_evaluate(tempt[var].values.reshape(-1),tempt.prob_pred_v2.squeeze(),best_threshold_v2)
            false_positive_v2.append(test_output_v2["false positive"])
            false_negative_v2.append(test_output_v2["false_negative"])

        else:
            false_positive_lexican.append(0)

            false_negative_v0.append(0)
            false_positive_v0.append(0)

            false_negative_v1.append(0)
            false_positive_v1.append(0)

            false_negative_v2.append(0)
            false_positive_v2.append(0)
            
    bar_width=0.25
    spacing=1.5
    
    index=np.arange(0, len(daily_vt)*spacing,spacing)
    plt.figure(figsize=(15, 8))

    bar0=plt.bar(index, false_positive_lexican, bar_width,  label="Lexican_search", color="blue", edgecolor="black")
    bar1=plt.bar(index+bar_width, false_positive_v0, bar_width,  label="Existing TFIDF Model", color="red", edgecolor="black")
    bar2=plt.bar(index+2*bar_width, false_positive_v1, bar_width,  label="Updated TFIDF Model", color="green", edgecolor="black")
    bar3=plt.bar(index+3*bar_width, false_positive_v2, bar_width,  label="Language Model", color="purple", edgecolor="black")

    plt.xlabel('Day', fontsize=14)
    plt.ylabel('Number of False Positive', fontsize=14)
    plt.title(f'Daily False Positive \n({subtitle})', fontsize=16)
    plt.grid(axis="y")

    plt.xticks(index+bar_width,daily_vt)
    plt.xticks(rotation=90)
    # plt.ylim(0.006,0.03)
    # Set the legend
    plt.legend()
    plt.tight_layout()
    plt.show()
    
    bar_width=0.25
    spacing=1.5
    index=np.arange(0, len(daily_vt)*spacing,spacing)
    plt.figure(figsize=(10, 4))

    bar1=plt.bar(index, false_negative_v0, bar_width,  label="Existing TFIDF Model", color="red", edgecolor="black")
    bar2=plt.bar(index+bar_width, false_negative_v1, bar_width,  label="Updated TFIDF Model", color="green", edgecolor="black")
    bar3=plt.bar(index+2*bar_width, false_negative_v2, bar_width,  label="Language Model", color="purple", edgecolor="black")

    plt.xlabel('Day', fontsize=14)
    plt.ylabel('Number of False Negative', fontsize=14)
    plt.title(f'Daily False Negative \n({subtitle})', fontsize=16)
    plt.grid(axis="y")

    plt.xticks(index+bar_width,daily_vt)
    plt.xticks(rotation=90)

    plt.yticks([0,1,2])
    plt.ylim(0,2)
    # Set the legend
    plt.legend()
    plt.tight_layout()
    plt.show()

In [ ]:
recall_precision_chart(email_data, var="target")

In [ ]:
false_psoitive_negative_chart(email_data)

In [ ]:
def false_psoitive_reduction(email_data, var="target"):
    false_negative_v0=[]
    # false_negative_v1=[]
    # false_negative_v2=[]
    false_positive_v0=[]
    # false_positive_v1=[]
    # false_positive_v2=[]

    false_positive_lexican=[]
    daily_vt=sorted(email_data.day.unique())

    for i,t in enumerate(daily_vt):
        tempt=email_data[email_data.day==t]    
        best_threshold_v0=tempt.best_threshold_v0.unique()[0]
        # best_threshold_v1=tempt.best_threshold_v1.unique()[0]
        # best_threshold_v2=tempt.best_threshold_v2.unique()[0]    

        if tempt[tempt[var]==1].shape[0]!=0:
            a, b=tempt[var].value_counts().tolist()    
            false_positive_lexican.append(a)
            test_output_v0=utils.model_evaluate(tempt[var].values.reshape(-1),tempt.prob_pred_v0.squeeze(),best_threshold_v0)
            false_positive_v0.append(test_output_v0["false positive"])
            false_negative_v0.append(test_output_v0["false_negative"])

#             test_output_v1=utils.model_evaluate(tempt[var].values.reshape(-1),tempt.prob_pred_v1.squeeze(),best_threshold_v1)
#             false_positive_v1.append(test_output_v1["false positive"])
#             false_negative_v1.append(test_output_v1["false_negative"])

#             test_output_v2=utils.model_evaluate(tempt[var].values.reshape(-1),tempt.prob_pred_v2.squeeze(),best_threshold_v2)
#             false_positive_v2.append(test_output_v2["false positive"])
#             false_negative_v2.append(test_output_v2["false_negative"])

        else:
            false_positive_lexican.append(0)

            false_negative_v0.append(0)
            false_positive_v0.append(0)

#             false_negative_v1.append(0)
#             false_positive_v1.append(0)

#             false_negative_v2.append(0)
#             false_positive_v2.append(0)
            

    bar_width=0.45
    spacing=1.5
    
    index=np.arange(0, len(daily_vt)*spacing,spacing)
    plt.figure(figsize=(15, 8))

    bar0=plt.bar(index, false_positive_lexican, bar_width,  label="Lexican_search", color="blue", edgecolor="black")
    bar1=plt.bar(index+bar_width, false_positive_v0, bar_width,  label="TFIDF Model", color="red", edgecolor="black")

    # Annotate with the percentage reduction
    for i, (v0, v1) in enumerate(zip(false_positive_lexican, false_positive_v0)):
        if v0!=0:
            reduction = ((v0 - v1) / v0) * 100
            plt.annotate(f'-{reduction:.1f}%', 
                         (index[i] + bar_width, v1 + 2), 
                         ha='center', va='bottom', 
                         fontsize=10, 
                         color='black')


    plt.xlabel('Day', fontsize=14)
    plt.ylabel('Number of False Positive', fontsize=14)
    plt.title('Daily False Positive \n(test_set=0901~0930)', fontsize=16)
    plt.grid(axis="y")

    plt.xticks(index+bar_width,daily_vt)
    plt.xticks(rotation=90)
    # plt.ylim(0.006,0.03)
    # Set the legend
    plt.legend()
    plt.tight_layout()
    plt.show()